In [ ]:
import pulp
import numpy as np

N = 10  # количество точек POI
B = 100  # ограничение по времени на маршрут

scores = np.random.rand(N)  # случайные оценки для каждой точки POI
travel_times = np.random.rand(N, N) * 10  # случайные времена перемещения между точками

prob = pulp.LpProblem("Maximize_POI_Score", pulp.LpMaximize)

x = pulp.LpVariable.dicts("x", ((i, j) for i in range(N) for j in range(N)), cat="Binary")
u = pulp.LpVariable.dicts("u", (i for i in range(N)), lowBound=2, upBound=N, cat="Integer")

# максимизация суммы оценок
prob += pulp.lpSum([x[i, j] * scores[i] for i in range(1, N-1) for j in range(2, N)])

# проверяем, что маршрут начинается с начальной точки и заканчивается конечной точкой
prob += pulp.lpSum([x[0, j] for j in range(1, N)]) == 1  # старт с точки 1
prob += pulp.lpSum([x[i, N-1] for i in range(N-1)]) == 1  # конец в точке N

for k in range(1, N-1):
    prob += pulp.lpSum([x[j, k] for j in range(N)]) == pulp.lpSum([x[k, j] for j in range(N)])

prob += pulp.lpSum([x[i, j] * travel_times[i][j] for i in range(N) for j in range(N)]) <= B

for i in range(1, N):
    for j in range(1, N):
        if i != j:
            prob += u[i] - u[j] + N * x[i, j] <= N - 1

prob.solve()
print("Статус решения:", pulp.LpStatus[prob.status])
print("Оптимальный маршрут:")
for i in range(N):
    for j in range(N):
        if pulp.value(x[i, j]) == 1:
            print(f"От {i} до {j}")

print("Максимальная сумма оценок:", pulp.value(prob.objective))
